In [ ]:
from gos import World, Neighborhood
from gos import visualization 
import matplotlib
import pandas as pd

In [ ]:
def file_path(name):
    """
    Shortcut function to get the relative path to the directory
    which contains the data.
    """
    return "./examples/multiscale-migration/data/%s" % name

In [ ]:
column_names = ["Name", "Code"]

def country_codes():
    """
    Build country rows from their names, ISO codes, and Numeric
    Country Codes.
    """
    cc = (
        pd.read_csv(
            file_path(
                "Country_List_ISO_3166_Codes_Latitude_Longitude.csv"),
            usecols=[0, 2, 3],
            index_col=1,
            keep_default_na=False))
    cc.columns = column_names
    return cc

def other_codes():
    other_codes = pd.read_csv(file_path("other.csv"), index_col=1)
    other_codes.columns = column_names[0:1]
    return other_codes

world = World(index=set(country_codes().index) | set(other_codes().index))
Neighborhood.update(country_codes().groupby("Alpha-3 code")["Name"].apply(list).to_dict())
Neighborhood.update(other_codes().groupby('ISO')["Name"].apply(list).to_dict())
Neighborhood.update(country_codes().groupby("Alpha-3 code")["Code"].apply(list).to_dict())

In [ ]:
population = pd.read_csv(file_path("newPOP.csv"))
population = population.set_index("Country")

world.update_neighborhoods(population)

In [ ]:
un_pd = pd.read_excel(
        file_path(
            "UN_MigrantStockByOriginAndDestination_2015.xlsx"
        ),
        skiprows=15
    )
un_pd = un_pd.set_index('Unnamed: 1')
un_pd = un_pd.iloc[0:275,7:250]

# TODO: Should we be using the UN numbers for this?
un_pd = un_pd.sort_index().fillna(1)
world.add_matrix("un", un_pd)

In [ ]:
visualization.matrix_plot(world.matrices["un"].iloc[:25, :25])

In [ ]:
visualization.map_plot(world.data["Population"])

In [ ]:
visualization.map_plot(
    world.data["Population"] - world.data["Population"]["USA"],
    title="Population compared to U.S."
)